In [1]:
from pyspark.sql import SparkSession
from pysparkling import *
import h2o

ss = SparkSession.builder.getOrCreate()
hc = H2OContext.getOrCreate()

Connecting to H2O server at http://192.168.0.126:54323 ... successful.


H2O_cluster_uptime:,11 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,1 month and 3 days
H2O_cluster_name:,sparkling-water-dwoodbridge_local-1614957303290
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"



Sparkling Water Context:
 * Sparkling Water Version: 3.32.0.4-1-3.0
 * H2O name: sparkling-water-dwoodbridge_local-1614957303290
 * cluster size: 1
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (0,192.168.0.126,54321)
  ------------------------

  Open H2O Flow in browser: http://192.168.0.126:54323 (CMD + click in Mac OSX)

    


## Mute the progress bar

In [2]:
h2o.no_progress()

In [3]:
adult_train_df = ss.read.parquet("../Data/spark-warehouse/adulttrain")
adult_train_h2o = hc.asH2OFrame(adult_train_df, "adult_train")
adult_train_h2o["label"] = adult_train_h2o["label"].asfactor()

adult_valid_df = ss.read.parquet("../Data/spark-warehouse/adultvalid")
adult_valid_h2o = hc.asH2OFrame(adult_valid_df, "adult_valid")
adult_valid_h2o["label"] = adult_valid_h2o["label"].asfactor()

predictors = adult_valid_h2o.names[:]
response = "label"
predictors.remove(response)

## AutoML - max_models = 3

In [5]:
from h2o.automl import H2OAutoML
model_automl = H2OAutoML(max_models = 3, seed = 1, nfolds = 5) # seed : Set a seed for reproducibility
model_automl.train(x=predictors,
                   y=response,
                   training_frame=adult_train_h2o,
                   validation_frame=adult_valid_h2o)


07:15:35.278: User specified a validation frame with cross-validation still enabled. Please note that the models will still be validated using cross-validation only, the validation frame will be used to provide purely informative validation metrics on the trained models.



## Return the best model's performance

In [6]:
model_automl.leader.auc(valid=True)

0.9243267461698755

In [7]:
ss.stop()